In [5]:
import numpy as np
import cv2
from PIL import Image

In [32]:
def showImg(img):
    pil_image=Image.fromarray(np.uint8(img))
    pil_image.show()

In [80]:
path = 'data/lena_gray.png'
img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
if img is None:
    print("Could not read image from path: " + path)

showImg(img)

In [81]:
l=5
G = img.copy()
gpImg = [G]
for i in range(l):
    G = cv2.pyrDown(G)
    gpImg.append(G)

gpImg.reverse()

In [84]:
showImg(gpImg[5])

In [91]:
lpImg = [gpImg[0]]
for i in range(l):
    GE = cv2.pyrUp(gpImg[i])
    L = cv2.subtract(gpImg[i+1], GE)
    lpImg.append(L)

In [90]:
showImg(lpImg[0])


In [92]:
ls_ = lpImg[0]
for i in range(1, l+1):
    ls_ = cv2.pyrUp(ls_)
    ls_ = cv2.add(ls_, lpImg[i])

In [93]:
showImg(ls_)
